In [11]:
INF = 1e9
Node = {}
Edge = {}
Part = {}
# def SendMessageTo(Tar,Value):

class HyperNode:
    def __init__(self,n_id):
        self.n_id = n_id
        self.dist = INF 
        self.edges = []
        
    def AddEdge(self,edge):
        self.edges.append(edge)
        
    def GetHyperEdges():
        return self.edges
    
    def GetValue(self):
        return self.dist
    
    def compute(self,msgs):
        if IsSource(self.v_id) : mindist = 0
        else : mindist = INF 
        mindist = min(msgs)
        if mindist < self.dist : 
            self.dist = mindist
            for hyperEdge in self.GetHyperEdges():
                for neibor in hyperEdge.GetHyperNodes():
                    SendMessageTo(neibor,mindist+hyperEdge.getValue())
                    
class HyperEdge:
    def __init__(self,e_id):
        self.e_id = e_id
        self.nodes = []
        
    def AddNode(self,node):
        self.nodes.append(node)
    
    def GetHyperNodes():
        return self.nodes
    
def solve():
    while 
        for n_id,node in Node.items():
            node.compute(msg[n_id])
        


def load_data():
    with open("./test_data/partition_method.txt") as file :
        for line in file:
            v_id,par_id = line[0:-1].split(" ")
            Part[v_id] = par_id
    
    with open("./test_data/github_vertex_info.txt") as file:
        for line in file:
            data = line[0:-1].split(" ")
            v_id = int(data[0])
            Node[v_id] = HyperNode(v_id)
            for e_id in data[1:]:
                Node[v_id].AddEdge(int(e_id))
                
    with open("./test_data/github_edge_info.txt") as file:
        for line in file:
            data = line[0:-1].split(" ")
            e_id = int(data[0])
            Edge[e_id] = HyperEdge(e_id)
            for n_id in data[1:]:
                Edge[e_id].AddNode(int(n_id))
            
load_data()
        
    

[1]
[2, 9346, 19334, 3871]
[3, 1957, 95, 4390, 6118, 7010, 3117, 7798, 8220, 8359, 1642, 5652, 10926, 4201, 6181, 102, 4071, 16005, 12439, 2848, 15723, 5194, 337, 2256, 5530, 3332, 2091, 964, 5269, 1407, 8537, 8251, 14945, 4284, 3994, 8465, 632, 425, 3341, 11017, 11849, 6964, 6410, 2394, 1671, 27115, 1577, 14167, 2992, 1422, 17238, 4541, 1836, 6972, 3691, 4711, 1393, 2301, 17136, 1404, 1053, 31095, 8946, 21754, 31674, 681, 3002, 3426, 83, 24, 4833, 7525, 4843, 4686, 442, 2242, 2312, 1691, 4328, 2040, 2545, 7862, 15953, 1069, 7937, 15924, 1293, 6945, 18770, 6975, 11203, 5911, 24343, 5847, 4331, 2965, 6377, 1281, 123, 9360, 955, 3553, 8995, 1440, 16842, 791, 264, 8408, 1676, 31557, 1758, 3512, 22142, 7069, 3286, 15293, 1153, 2476, 4466, 4840, 14898, 8112, 1320, 365, 5190, 1356, 14818, 12495, 4931, 3134, 571, 2573, 3945, 4927, 10032, 20039, 3305, 1753, 31972, 1705, 121, 3287, 39147, 3939, 2436, 7098, 29, 5875, 258, 1090, 1647, 13258, 5976, 5779, 5771, 1872, 5795, 10378, 15083, 3178, 15544

[1212, 20, 3115, 7455, 1342, 426, 5375, 3248, 20140, 379, 2823, 18935, 298, 1882, 69, 515, 11516, 6379, 10955, 4400, 3020, 964, 975, 13985, 8751, 4589, 7734, 4090, 322, 4390, 2374, 1285, 20076, 3760, 1544, 5819, 363, 22513, 8588, 11336, 83, 10919, 3198, 20950, 5749, 171, 389]
[1214, 4357, 8365, 57, 9998, 1473, 17493, 18591, 46, 5288, 12835, 10298, 3384, 12698, 8066, 15641, 1034, 3683, 5690, 1433, 2596, 6031, 1680, 201, 26908, 7859, 935, 4147, 24221, 13049, 3040, 1604, 5322, 71, 6848, 8409, 1567, 1665, 2071, 7165, 5610, 3868, 13955, 1436, 471, 17601]
[1215, 2356, 3108, 3642, 3894, 1080, 7245, 9393, 9487, 4808, 3340, 7838, 12272, 12281, 12742, 13523, 15154, 15275, 6669, 17007, 4928, 6844, 10041, 19442, 16150, 432, 11793, 20443, 2159, 7376, 2782, 22441, 1800, 4092, 22801, 4500, 5567, 382, 1419, 12476, 4562, 11778, 25970, 446, 7627, 1753, 16316, 12387, 3648, 367, 6427, 27909, 28010, 962, 87, 7212, 5366, 5621, 4957, 4869, 3282, 1947, 8183, 2312, 32617, 1691, 12102, 15781, 32975, 33010, 4166

[2023, 2978, 5191, 5196, 376, 9735, 10197, 11124, 1419, 15162, 12519, 8647, 187, 71, 9742, 9956, 10442, 20581, 3939, 18914, 20976, 19457, 6613, 6250, 21514, 15826, 2758, 22022, 22355, 22367, 501, 23349, 24618, 26134, 2613, 11536, 9388, 4676, 28586, 14818, 28876, 28978, 29332, 4817, 7168, 11668, 5451, 686, 31718, 3216, 31792, 2558, 32155, 32393, 16724, 5454, 34099, 10449, 1180, 4373, 23633, 23288, 9405, 11283, 3916, 5048, 427, 14251, 3620, 2117, 1980, 11053, 35781, 5594, 36615, 18079, 562, 1555, 41556, 41642, 5079, 42115, 1676, 31167, 3881, 42594, 983, 6904, 627, 43033, 478, 20029, 11933, 43811, 30535, 38181, 13967, 2212, 29343, 42098, 12619, 6114, 9855, 17558, 9954, 23428, 5914, 46410, 22175, 22609, 28954, 5604, 10520, 33564, 14845, 41676, 8290, 43726, 29123, 39696, 11478, 34758, 3090, 13741, 31526, 22859, 21059, 34952, 4639, 49859, 9495, 537, 2434, 50044, 43341, 13874, 32271, 263, 1625, 15611, 6547, 7663, 47977, 14320, 13121, 6286, 18042, 18850, 35673, 5833, 8513, 17002, 42060, 4669, 

[2988, 10586, 11434, 3404, 9970, 13695, 24991, 13046, 6850]
[2989]
[565, 3872, 4914, 6740, 8166, 427, 926, 445, 10650, 20749, 11069, 5672, 4454, 4504, 13686, 2119, 3108, 5205, 3114, 14870, 5604, 1791, 1337, 3243, 9575, 13250, 544, 2100, 19379, 1810, 4733, 387, 4225, 1672, 6600, 41570, 298, 20701, 3689, 1393, 2160, 16927, 922, 10600, 222, 12709, 3438, 7229, 2172, 3585, 226, 7179, 929, 27603, 2472, 10181, 2122, 23567, 83, 537, 8093, 2169, 5541, 6022, 12664, 7068, 1574, 15320]
[2990]
[572, 646, 4721, 325, 7772, 12489, 2067, 15043, 1472, 1211, 518, 31, 1407, 4862, 12439, 627, 351, 313, 741, 2138, 11753, 1882, 5571, 1208, 506, 1570, 16, 2204, 6915, 2494, 1170, 1808, 1549, 2432, 3287, 3168, 2056, 655, 12068, 12184, 349, 744, 931, 3357, 4231, 4000, 215, 1614, 2547, 2222, 89, 6940, 987, 4022, 363, 1631, 2521, 347, 1666, 13596, 2024, 4661, 2148, 2472, 5334, 3728, 446, 964, 4230, 1656, 1753, 1828, 1691, 272, 1151, 5742, 501, 7492, 1979, 315, 3120]
[2993, 2037, 28658, 28415]
[673, 2897, 3034, 956

[3094, 51065, 31586]
[3095, 7515, 3769, 13386, 14575, 9992, 13714, 25506, 4232, 12151, 30641, 13783, 33578, 18467, 2169, 12355, 21464, 40444, 1405, 5776, 1560, 11976, 563, 9529, 1452, 5272, 9467, 24747, 486, 4900, 12386, 56499]
[3096, 3792, 723, 7431, 8114, 9474, 9953, 2304, 10970, 2066, 7780, 3152, 71, 427, 6431, 9597, 8066, 5057, 11396, 349, 1231, 17072, 1644, 7134, 1146, 47, 16992, 8049, 6270, 13955, 1961, 3020, 1808, 7405, 2050, 3047, 497, 12579, 11914, 8565, 4827, 7181, 1030, 1665, 27463, 6970, 14027, 7120, 11041, 1614, 1045, 935, 11847, 16988, 4273, 5274, 21574, 4965, 1679, 6206, 31538, 2614, 5447, 7596, 18114, 3174, 21855, 33360, 1648, 6302, 33768, 2085, 1180, 7725, 888, 9123, 142, 627, 12709, 1523, 17828, 5087, 1715, 4081, 3374, 9692, 2149, 2284, 7408, 8840, 5266, 86, 1822, 7796, 1241, 16, 4638, 1475, 16387, 8608, 4128, 2526, 4343, 3593, 20235, 46, 1876, 798, 704, 4974, 9552, 5610, 1837, 16307, 1047, 376, 44663, 23053, 6844, 3697, 215, 8907, 1153, 702, 10336, 3040, 6715, 3235, 

...

[40682]
[23140]
[54310]
[7804]
[54314]
[2973]
[628]
[54317]
[3817]
[11502]
[6486]
[12664]
[54322]
[52637]
[2008]
[54324]
[8072]
[34508]
[26785]
[10359]
[54328]
[8346]
[7163]
[1851]
[4681]
[9548]
[4116]
[54330]
[1833]
[33825]
[4169]
[38793]
[54331]
[54332]
[6523]
[8722]
[18845]
[1709]
[24010]
[54334]
[9896]
[2041]
[3419]
[46394]
[18670]
[3558]
[54338]
[8443]
[6105]
[19275]
[864]
[3577]
[6250]
[4158]
[44750]
[54341]
[10592]
[7383]
[45580]
[18365]
[8886]
[17103]
[53396, 2207]
[24661]
[45327]
[35168]
[18508]
[5301]
[100]
[54343]
[15711]
[44751]
[856]
[8748]
[12664]
[361]
[24134]
[24558]
[4710]
[6978]
[5272]
[6899, 24054]
[22892]
[3625]
[54346]
[49880]
[7379]
[37426]
[35816]
[35078]
[21057]
[9374]
[9053]
[54348]
[3243, 6890]
[18285]
[2817]
[32098]
[6876]
[44]
[11411]
[54352]
[156]
[2317]
[14677]
[12898]
[7351]
[4629]
[697]
[17693]
[13185]
[15596]
[1124]
[23722]
[4609]
[54356]
[43665]
[18227]
[4560]
[32063]
[12532]
[1734]
[10068]
[36644]
[54359]
[29705]
[6117]
[13175]
[7380]
[42120]
[152]
[1

[8193]
[30652]
[10285]
[1475]
[198]
[13869]
[4035]
[10724]
[3351]
[25721]
[30992]
[55037]
[2817]
[55038]
[2433]
[2787]
[8092]
[18495]
[2044]
[1725]
[26465]
[14510]
[1229, 1374]
[9648]
[1143]
[17348]
[19502]
[20675]
[19247]
[55039]
[16989]
[12904]
[30305]
[9548]
[55040]
[6020]
[17726]
[35855]
[55041]
[6259]
[55042]
[55043]
[40485]
[8012]
[124]
[4303, 8749]
[46104]
[38244]
[44559]
[26869]
[142]
[23188]
[13732]
[2433]
[42657]
[7255]
[55048]
[43195]
[55049]
[643]
[14227]
[19049]
[1160]
[37101]
[1676]
[3342]
[7322]
[55050]
[4651, 26304]
[55051]
[55052]
[3984]
[1742]
[2817]
[1413]
[7403]
[13421]
[15739]
[28486]
[47429]
[14485]
[46383]
[48528]
[260]
[23760]
[138]
[10532]
[28196]
[7728]
[39205]
[3473]
[55056]
[14054]
[10486]
[37688]
[4362]
[4101]
[55058]
[2014]
[28229]
[55061]
[3341]
[11810]
[26446]
[19039]
[55064]
[15976]
[34265]
[55069]
[55070]
[34552]
[17275]
[20821]
[3600]
[52461]
[55072]
[1782]
[7780]
[55073]
[8259]
[11143]
[15219]
[25552]
[47860]
[55074]
[14580]
[26643]
[37755]
[16989]
[

[10852]
[25564]
[15018]
[11282]
[15617]
[15025]
[2999]
[47563]
[27626]
[10400]
[1619]
[25080]
[26005]
[4212]
[663]
[393]
[12606]
[11828]
[32106]
[11797]
[9132]
[7656]
[34122]
[14159]
[55382]
[12708]
[53405]
[55383]
[50014]
[34279]
[9261]
[42742]
[10051]
[55385]
[23303]
[55386]
[12966]
[35340]
[15765]
[2829]
[3728]
[22201]
[1164]
[8431]
[13753]
[22367]
[55389]
[9719]
[12915]
[6127]
[2480]
[2632]
[28532]
[35250]
[8023]
[48480]
[7192]
[55391]
[24849]
[1170]
[39763]
[55392]
[810]
[20870]
[2235]
[34063]
[17425]
[2632]
[19535]
[55394]
[24306]
[18394]
[35909]
[7331]
[2500]
[2256]
[1523]
[27193]
[55396]
[34731]
[55397]
[9555]
[3239]
[13982]
[55398]
[4138]
[43666]
[55400]
[2814]
[17066]
[6300]
[3403]
[41396]
[55401]
[47740]
[55402]
[868]
[17965]
[13677]
[15289]
[36458]
[55405]
[10509]
[5967]
[10862]
[9896]
[2376]
[55406]
[1626]
[46132]
[42599]
[17617]
[26743]
[4748]
[6613]
[25510]
[19971]
[24737]
[4359]
[8709]
[55408]
[16462]
[37]
[10650]
[7163]
[16711]
[6267]
[9394]
[10315]
[4613]
[4926]
[5541

[44158]
[56059]
[14230]
[1728]
[16100]
[1636]
[11856]
[8395]
[8884]
[1588]
[33918]
[2904]
[56062]
[3565]
[20889]
[7881]
[15170]
[926]
[16303]
[6151]
[47121]
[3574]
[5011]
[56063]
[20678]
[32274]
[32491]
[53187]
[8644]
[42369]
[21323]
[44557]
[3800]
[12503]
[42074]
[27769]
[52020]
[8556]
[24486]
[2522]
[56065]
[686]
[9781]
[13753]
[25028]
[298]
[35395]
[5894]
[7661]
[31621]
[2846]
[17031]
[1487]
[22739]
[21033]
[1348]
[9768]
[1183]
[28671]
[8904]
[1225]
[45891]
[25028]
[39822]
[2089]
[56072]
[387]
[56074]
[56077]
[56078]
[4169]
[972]
[3414]
[20630]
[16165]
[1167]
[8886]
[22]
[45082]
[11428]
[14483]
[21405]
[13948]
[6199]
[6492]
[9262]
[56082]
[56083]
[22903]
[3129]
[5436]
[921]
[7403]
[5894]
[2189]
[13015]
[56084]
[35626]
[8007]
[532]
[49214]
[568]
[6456]
[6175]
[34624]
[7895]
[39346]
[29155]
[2643]
[35130]
[7241]
[12253]
[56087]
[23698]
[12658]
[12]
[56089]
[54762]
[56091]
[20526]
[56094]
[17786]
[17951]
[18148]
[32331]
[16497]
[56098]
[9896]
[5342]
[9935]
[5207]
[2650]
[56099]
[13979]

[1673]
[31620]
[56383]
[8953]
[20545]
[14376]
[8801]
[36103]
[3414]
[44862]
[20768]
[8595]
[7599]
[41928]
[40083]
[56384]
[56385]
[56007]
[5833]
[56386]
[56387]
[8326]
[22347]
[1212]
[8013]
[7441]
[21777]
[5941]
[23740]
[7052]
[955]
[24217]
[25415]
[21559]
[24494]
[20675]
[19976]
[56392]
[532]
[17168]
[6694]
[38682]
[8953]
[2311]
[2002]
[7087]
[16826]
[49285]
[40872]
[40989]
[56393]
[56394]
[1313]
[50354]
[20134]
[2048]
[56395]
[6608]
[10211]
[20366]
[4104]
[20790]
[1766]
[56398]
[4925]
[56399]
[26082]
[19254]
[5638]
[908]
[33118]
[56401]
[6784]
[3108]
[9570]
[2969]
[43271]
[1934]
[47884]
[8639]
[16356]
[6521]
[3863]
[20052]
[2976]
[25658]
[11637]
[4741]
[19154]
[4273]
[4283]
[14134]
[8802]
[15292]
[8940]
[56407]
[2751]
[7172]
[56408]
[12902]
[20242]
[14436]
[56410]
[6648]
[7716]
[6050]
[6697]
[45115]
[34690]
[28416]
[48528]
[7413]
[52835]
[20806]
[1762]
[2947]
[34386]
[7011]
[18634]
[15408]
[45670]
[272]
[56417]
[42511]
[49751]
[4240]
[7068]
[2454]
[56419]
[17343]
[3185]
[5139]
[39469